In [1]:
!pip install crewai==1.7.2

In [2]:
!pip install \
langchain==0.1.20 \
langchain-community==0.0.38 \
langchain-openai==0.0.8 \
pypd

In [7]:
!pip install crewai-tools


  Using cached crewai_tools-1.7.2-py3-none-any.whl.metadata (11 kB)
  Using cached beautifulsoup4-4.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pymupdf-1.26.7-cp310-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached python_docx-1.2.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached tiktoken-0.8.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached youtube_transcript_api-1.2.3-py3-none-any.whl.metadata (24 kB)
  Using cached lxml-6.0.2-cp311-cp311-win_amd64.whl.metadata (3.7 kB)
Using cached crewai_tools-1.7.2-py3-none-any.whl (766 kB)
Using cached beautifulsoup4-4.13.5-py3-none-any.whl (105 kB)
Using cached docker-7.1.0-py3-none-any.whl (147 kB)
Using cached pymupdf-1.26.7-cp310-abi3-win_amd64.whl (18.4 MB)
Using cached python_docx-1.2.0-py3-none-any.whl (252 kB)
Using cached tiktoken-0.8.0-cp311-cp311-win_amd64.whl (884 kB)
Using cached youtube_transcript_api-1.2.3-py3-none-any.whl (485 kB)
Using cached

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [9]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)


docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)


inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

In [11]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)
print(result.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e92f72c9-0518-451d-bf94-84da669fdc2f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: Thought: I need to check the documentation on setting up a Crew and specifically how to add memory    │
│  to a crew. The best place to find this information is the crewAI documentation at                              │
│  https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/. I will use the Read website content tool to   │
│  gather detailed and complete instructions about adding memory to a crew.                                       │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "description": "Read the documentation on creating a Crew and specifically how to add memory to the crew."   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The following text is scraped website content:                                                                 │
│                                                                                                                 │
│  CrewAI Documentation - CrewAI Skip to main content CrewAI home page English Search... ⌘ K Start Cloud Trial    │
│  crewAIInc / crewAI crewAIInc / crewAI Search... Navigation Welcome CrewAI Documentation Home Documentation     │
│  AOP API Reference Examples Changelog Website Forum Blog CrewGPT Welcome CrewAI Documentation Welcome CrewAI    │
│  Documentation Copy page Build collaborative AI agents, crews, and flows — production ready from day one. Copy  │
│  page ​ Ship multi‑agent systems with confidence Design agents, orchestrate crews, and automate flows with       │
│  guardrails, memory, knowledge, and observability baked in. Get started View changelog API Reference            │
│  ​ Get started                                                                                                   │
│  Introduction Overview of CrewAI concepts, architecture, and what you can build with agents, crews, and flows.  │
│  Installation Install via uv , configure API keys, and set up the CLI for local development. Quickstart Spin    │
│  up your first crew in minutes. Learn the core runtime, project layout, and dev loop.                           │
│  ​ Build the basics                                                                                              │
│  Agents Compose agents with tools, memory, knowledge, and structured outputs using Pydantic. Includes           │
│  templates and best practices. Flows Orchestrate start/listen/router steps, manage state, persist execution,    │
│  and resume long-running workflows. Tasks & Processes Define sequential, hierarchical, or hybrid processes      │
│  with guardrails, callbacks, and human-in-the-loop triggers.                                                    │
│  ​ Enterprise journey                                                                                            │
│  Deploy automations Manage environments, redeploy safely, and monitor live runs directly from the Enterprise    │
│  console. Triggers & Flows Connect Gmail, Slack, Salesforce, and more. Pass trigger payloads into crews and     │
│  flows automatically. Team management Invite teammates, configure RBAC, and control access to production        │
│  automations.                                                                                                   │
│  ​ What’s new                                                                                                    │
│  Triggers overview Unified overview for Gmail, Drive, Outlook, Teams, OneDrive, HubSpot, and more — now with    │
│  sample payloads and crews. Integration tools Call existing CrewAI automations or Amazon Bedrock Agents         │
│  directly from your cre...                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hello Andrew,                                                                                                  │
│                                                                                                                 │
│  Thanks so much for reaching out to crewAI! I’m excited to help you get your Crew set up with memory, which is  │
│  a fantastic feature that lets your agents keep track of conversations, recall previous interactions, and       │
│  maintain important context to make your workflows smarter and more dynamic.                                    │
│                                                                                                                 │
│  Here’s a friendly but detailed walkthrough on how to add memory to your Crew, based on our current docs and    │
│  best practices:                                                                                                │
│                                                                                                                 │
│  1. **What Are Crews and Agents?**                                                                              │
│     A Crew in crewAI is basically a team of Agents working together to handle complex tasks. Each Agent can be  │
│  customized with different abilities, and one of the most powerful is memory — this lets the Agent remember     │
│  info between interactions.                                                                                     │
│                                                                                                                 │
│  2. **Memory is an Agent Feature**                                                                              │
│     Memory isn’t added directly to the Crew as a whole—it’s something you configure at the Agent level. This    │
│  lets each Agent hold onto conversational history or other relevant data either short-term or long-term,        │
│  depending on your needs.                                                                                       │
│                                                                                                                 │
│  3. **How to Add Memory to an Agent**                                                                           │
│     When you define an Agent for your Crew, you specify which memory type it will use by including it in the    │
│  Agent’s configuration. We support various memory types, such as in-memory buffers for quick conversational     │
│  context, vector stores for semantic search, or even persistent custom memories linking to databases.           │
│                                                                                                                 │
│     Here’s a concept snippet to illustrate:                                                                     │
│     ```python                                                                                                   │
│     from crewai import Agent, MemoryType                                                                        │
│                                                                                                                 │
│     # Create an agent with conversation buffer memory enabled                                                   │
│     my_agent = Agent(                                  

Hello Andrew,

Thanks so much for reaching out to crewAI! I’m excited to help you get your Crew set up with memory, which is a fantastic feature that lets your agents keep track of conversations, recall previous interactions, and maintain important context to make your workflows smarter and more dynamic.

Here’s a friendly but detailed walkthrough on how to add memory to your Crew, based on our current docs and best practices:

1. **What Are Crews and Agents?**  
   A Crew in crewAI is basically a team of Agents working together to handle complex tasks. Each Agent can be customized with different abilities, and one of the most powerful is memory — this lets the Agent remember info between interactions.

2. **Memory is an Agent Feature**  
   Memory isn’t added directly to the Crew as a whole—it’s something you configure at the Agent level. This lets each Agent hold onto conversational history or other relevant data either short-term or long-term, depending on your needs.

3. **How to A

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 80f7bca0-52a5-4215-814a-8ca23a9f986c                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e92f72c9-0518-451d-bf94-84da669fdc2f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hello Andrew,                                                                                    │
│                                                                                                                 │
│  Thanks so much for reaching out to crewAI! I’m excited to help you get your Crew set up with memory, which is  │
│  a fantastic feature that lets your agents keep track of conversations, recall previous interactions, and       │
│  maintain important context to make your workflows smarter and more dynamic.                                    │
│                                                                                                                 │
│  Here’s a friendly but detailed walkthrough on how to add memory to your Crew, based on our current docs and    │
│  best practices:                                                                                                │
│                                                                                                                 │
│  1. **What Are Crews and Agents?**                                                                              │
│     A Crew in crewAI is basically a team of Agents working together to handle complex tasks. Each Agent can be  │
│  customized with different abilities, and one of the most powerful is memory — this lets the Agent remember     │
│  info between interactions.                                                                                     │
│                                                                                                                 │
│  2. **Memory is an Agent Feature**                                                                              │
│     Memory isn’t added directly to the Crew as a whole—it’s something you configure at the Agent level. This    │
│  lets each Agent hold onto conversational history or other relevant data either short-term or long-term,        │
│  depending on your needs.                                                                                       │
│                                                                                                                 │
│  3. **How to Add Memory to an Agent**                                                                           │
│     When you define an Agent for your Crew, you specify which memory type it will use by including it in the    │
│  Agent’s configuration. We support various memory types, such as in-memory buffers for quick conversational     │
│  context, vector stores for semantic search, or even persistent custom memories linking to databases.           │
│                                                                                                                 │
│     Here’s a concept snippet to illustrate:                                                                     │
│     ```python                                                                                                   │
│     from crewai import Agent, MemoryType                                                                        │
│                                                                                                                 │
│     # Create an agent with conversation buffer memory 

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯